<a href="https://colab.research.google.com/github/LorenzoLaCorte/ML-basics-python/blob/main/Deep_Learning%2C_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducing Libraries
Main Deep Learning and Neural Network Libraries are:

* https://keras.io/ -> Easy to understand, it is implemented at an higher level of abstraction.
* https://pytorch.org/ -> For more Advance Users
* https://www.fast.ai/ -> For players

In [13]:
!nvidia-smi # this utility allows administrators to query GPU device state

Mon Nov 28 21:43:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    28W /  70W |   1338MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Keras

In [14]:
import numpy as np
from tensorflow import keras # we use the easier library
from tensorflow.keras import layers

In [15]:
# setting hyperparameters
classes = 10
shap = (28,28,1) # input of the model

(xl,yl), (xt,yt) = keras.datasets.mnist.load_data() # importing dataset
xl = xl.astype("float32") / 255.0
xt = xt.astype("float32") / 255.0

In [16]:
print("Shape of xl before expansion: " + str(np.shape(xl)))
xl = np.expand_dims(xl,-1)
xt = np.expand_dims(xt,-1)
print("Shape of xl after expansion: " + str(np.shape(xl)))

print("\nShape of yl before conversion: " + str(np.shape(yl)))
# we convert output as binary matrix representation where the class axis is placed last
yl = keras.utils.to_categorical(yl,classes)
yt = keras.utils.to_categorical(yt,classes) 
print("Shape after conversion: " + str(np.shape(yl)))

Shape of xl before expansion: (60000, 28, 28)
Shape of xl after expansion: (60000, 28, 28, 1)

Shape of yl before conversion: (60000,)
Shape after conversion: (60000, 10)


## Feature extraction with a Sequential model
A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

Once a Sequential model has been built, it behaves like a Functional API model. This means that every layer has an input and output attribute. These attributes can be used to do neat things, like quickly creating a model that extracts the outputs of all intermediate layers in a Sequential model.

We use **Relu** as activation function.

In [17]:
model = keras.Sequential(
    [
        keras.Input(shap),
        layers.Conv2D(32,kernel_size=(3,3),activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(.5), # w M w
        layers.Dense(classes,activation="softmax")
    ]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_1 (Dropout)         (None, 1600)             

Once the model is created, we can:
* config the model with losses and metrics with model.compile(), 
* train the model with model.fit(),
* or use the model to do prediction with model.predict().

In [18]:
model.compile(loss = "categorical_crossentropy", # loss function
              optimizer = "adam", 
              metrics = ["accuracy"]) # list of metrics to be evaluated by the model during training and testing.

epochs = 15 # number of iterations over the entire x and y data to train the model
batch_size = 128 # number of samples per gradient update

model.fit(xl,
          yl, 
          batch_size = batch_size,
          epochs = epochs,
          validation_split = 0.1) # fraction of the training data to be used as validation data

Epoch 1/15
422/422 [==============================] - 10s 6ms/step - loss: 0.3571 - accuracy: 0.8917 - val_loss: 0.0813 - val_accuracy: 0.9795
Epoch 2/15
422/422 [==============================] - 2s 5ms/step - loss: 0.1095 - accuracy: 0.9668 - val_loss: 0.0577 - val_accuracy: 0.9838
Epoch 3/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0835 - accuracy: 0.9740 - val_loss: 0.0469 - val_accuracy: 0.9880
Epoch 4/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0702 - accuracy: 0.9786 - val_loss: 0.0428 - val_accuracy: 0.9873
Epoch 5/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0603 - accuracy: 0.9811 - val_loss: 0.0403 - val_accuracy: 0.9897
Epoch 6/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0543 - accuracy: 0.9826 - val_loss: 0.0356 - val_accuracy: 0.9907
Epoch 7/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0516 - accuracy: 0.9841 - val_loss: 0.0350 - val_accuracy: 0.9915
Epoch

We can now **evaluate** the accuracy of this model (expected above 99%)

In [ ]:
acc = model.evaluate(xt,yt) # input and target data
print(acc[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.0227 - accuracy: 0.9925
0.9925000071525574


And confirm our results through a **confusion matrix**

In [ ]:
from sklearn.metrics import confusion_matrix
yp = model.predict(xt)
yp = yp.argmax(1)
yt = yt.argmax(1)
confusion_matrix(yt,yp)

313/313 [==============================] - 1s 2ms/step


array([[ 978,    0,    0,    0,    0,    0,    0,    1,    1,    0],
       [   0, 1133,    1,    0,    0,    0,    1,    0,    0,    0],
       [   1,    1, 1023,    0,    1,    0,    1,    3,    2,    0],
       [   0,    0,    2, 1003,    0,    3,    0,    0,    2,    0],
       [   0,    0,    0,    0,  979,    0,    0,    0,    0,    3],
       [   0,    0,    0,    6,    0,  883,    1,    0,    1,    1],
       [   3,    2,    0,    0,    2,    2,  948,    0,    1,    0],
       [   0,    3,    5,    2,    0,    1,    0, 1015,    1,    1],
       [   2,    0,    1,    0,    1,    1,    0,    0,  967,    2],
       [   0,    0,    0,    0,    4,    5,    1,    2,    1,  996]])